In [1]:
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import LSTM, Dense
import numpy as np

In [2]:
batch_size = 61
epochs = 100
latent_dim = 256
num_sample = 10000
data_path = './cmn-eng/cmn.txt'

In [3]:
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

In [4]:
input_texts = []
target_texts = []
input_chars = set()
target_chars = set()

for line in lines[:min(len(lines)-1, num_sample)]:
    input_text, target_text, _ = line.split('\t')
    target_text = '\t' + target_text + '\n'
    
    input_texts.append(input_text)
    target_texts.append(target_text)
    
    for c in input_text:
        if c not in input_chars:
            input_chars.add(c)
            
    for c in target_text:
        if c not in target_chars:
            target_chars.add(c)
        
        
    

In [5]:
input_chars = sorted(list(input_chars))
target_chars = sorted(list(target_chars))

In [6]:
num_encoder_tokens = len(input_chars)
num_decoder_tokens = len(target_chars)
max_encoder_seq_length = max([len(l) for l in input_texts]) + 1
max_decoder_seq_length = max([len(l) for l in target_texts]) + 1

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 73
Number of unique output tokens: 2577
Max sequence length for inputs: 31
Max sequence length for outputs: 23


In [7]:
input_token_index = {c: i for i, c in enumerate(input_chars)}
input_index_token = {i: c for i, c in enumerate(input_chars)}

target_token_index = {c: i for i, c in enumerate(target_chars)}
target_index_token = {i: c for i, c in enumerate(target_chars)}


encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for j, c in enumerate(input_text):
        encoder_input_data[i, j, input_token_index[c]] = 1.
    
    encoder_input_data[i, j+1:, input_token_index[' ']] = 1.
        
    # h  e l l o \n 
    # \t h e l l o \n
    for j, c in enumerate(target_text):
        decoder_input_data[i, j, target_token_index[c]] = 1.
        
        if j > 0:
            decoder_target_data[i, j-1, target_token_index[c]] = 1.
        
    decoder_input_data[i, j+1, target_token_index[' ']] = 1.
    decoder_target_data[i, j, target_token_index[' ']] = 1.

In [8]:
def reconstruct_text(input_data):
    for i in range(0, 5):
        input_text = ''
    
        for j in input_data[i]:
            pos = np.argmax(j)
            input_text += target_index_token[pos]
            
        print(input_text)

reconstruct_text(decoder_input_data)    
reconstruct_text(decoder_target_data)    

	嗨。
 																		
	你好。
 																	
	你用跑的。
 															
	等等！
 																	
	你好。
 																	
嗨。
 																			
你好。
 																		
你用跑的。
 																
等等！
 																		
你好。
 																		


In [9]:
# Model

encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_lstm = LSTM(latent_dim, return_state=True)
_, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_state=True, return_sequences=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=batch_size, epochs=epochs, validation_split=0.2)
model.save('seq2seq.h5')

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 73)]   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 2577)] 0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 256), (None, 337920      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  2902016     input_2[0][0]                    
                                                                 lstm[0][1]                   

Epoch 44/100
8000/8000 [==============================] - 110s 14ms/sample - loss: 0.5186 - accuracy: 0.3065 - val_loss: 1.7980 - val_accuracy: 0.2263
Epoch 45/100
8000/8000 [==============================] - 112s 14ms/sample - loss: 0.5059 - accuracy: 0.3086 - val_loss: 1.8271 - val_accuracy: 0.2224
Epoch 46/100
8000/8000 [==============================] - 113s 14ms/sample - loss: 0.4952 - accuracy: 0.3111 - val_loss: 1.8286 - val_accuracy: 0.2248
Epoch 47/100
8000/8000 [==============================] - 110s 14ms/sample - loss: 0.4845 - accuracy: 0.3134 - val_loss: 1.8436 - val_accuracy: 0.2262
Epoch 48/100
8000/8000 [==============================] - 112s 14ms/sample - loss: 0.4736 - accuracy: 0.3149 - val_loss: 1.8472 - val_accuracy: 0.2232
Epoch 49/100
8000/8000 [==============================] - 111s 14ms/sample - loss: 0.4634 - accuracy: 0.3174 - val_loss: 1.8432 - val_accuracy: 0.2258
Epoch 50/100
8000/8000 [==============================] - 130s 16ms/sample - loss: 0.4529 - ac

8000/8000 [==============================] - 161s 20ms/sample - loss: 0.1553 - accuracy: 0.3786 - val_loss: 2.1388 - val_accuracy: 0.2226
Epoch 99/100
8000/8000 [==============================] - 162s 20ms/sample - loss: 0.1517 - accuracy: 0.3796 - val_loss: 2.1601 - val_accuracy: 0.2205
Epoch 100/100
8000/8000 [==============================] - 161s 20ms/sample - loss: 0.1471 - accuracy: 0.3807 - val_loss: 2.1559 - val_accuracy: 0.2215


In [10]:
# INFERENCE

# ENCODER
encoder_model = Model(encoder_inputs, encoder_states)
encoder_model.summary()

# DECODER
decoder_state_inputs = [Input(shape=(latent_dim,)), Input(shape=(latent_dim,))]
decoder_outputs, next_h, next_c = decoder_lstm(decoder_inputs, initial_state=decoder_state_inputs)
next_states = [next_h, next_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_state_inputs,
    [decoder_outputs] + next_states)

decoder_model.summary()





Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, 73)]        0         
_________________________________________________________________
lstm (LSTM)                  [(None, 256), (None, 256) 337920    
Total params: 337,920
Trainable params: 337,920
Non-trainable params: 0
_________________________________________________________________
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, 2577)] 0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 256)]        0                                            
___

In [11]:
def decode_sequence(input_seq):
    states = encoder_model.predict(input_seq)
    
    target_seq = np.zeros((1,1,num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.
    
    stop = False
    decoded_text = ''
    while not stop:
        o, h, c = decoder_model.predict([target_seq] + states)
        
        sampled_token_index = np.argmax(o[0,-1,:])
        sampled_char = target_index_token[sampled_token_index]
        decoded_text += sampled_char
        
        if sampled_char == '\n' or len(decoded_text) > max_decoder_seq_length:
            stop = True
            
        target_seq = np.zeros((1,1,num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.
        
        states = [h, c]
        
    return decoded_text
        

for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)    
        

-
Input sentence: Hi.
Decoded sentence: 嗨。

-
Input sentence: Hi.
Decoded sentence: 嗨。

-
Input sentence: Run.
Decoded sentence: 你用跑的。

-
Input sentence: Wait!
Decoded sentence: 等等！

-
Input sentence: Hello!
Decoded sentence: 你好。

-
Input sentence: I try.
Decoded sentence: 让我来。

-
Input sentence: I won!
Decoded sentence: 我赢了。

-
Input sentence: Oh no!
Decoded sentence: 不会吧。

-
Input sentence: Cheers!
Decoded sentence: 嗨！

-
Input sentence: Got it?
Decoded sentence: 你懂了吗？

-
Input sentence: He ran.
Decoded sentence: 他跑了。

-
Input sentence: Hop in.
Decoded sentence: 跳进来。

-
Input sentence: I lost.
Decoded sentence: 我迷失了。

-
Input sentence: I quit.
Decoded sentence: 我迷失了。

-
Input sentence: I'm OK.
Decoded sentence: 我沒事。

-
Input sentence: Listen.
Decoded sentence: 听着。

-
Input sentence: No way!
Decoded sentence: 没门！

-
Input sentence: No way!
Decoded sentence: 没门！

-
Input sentence: Really?
Decoded sentence: 你确定？

-
Input sentence: Try it.
Decoded sentence: 试试吧。

-
Input sentence: We try